# Taxonomy classification

**Overview:**<br>
[1. Setup](#setup)<br>
[2. Taxonomy assignment](#tax_assignment_main)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Reference database construction](#ref_db)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Training taxonomy classifier](#train_classifier)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3 Taxonomy assignment](#tax_assignment)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.4 Taxonomy visualization](#tax_visualization)<br>

<a id='setup'></a>

## 1. Setup

The cell below will import all the packages required in the downstream analyses as well as set all the necessary variables and data paths.

In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline

# location of the data and all the results produced by this notebook 
data_dir = 'project_data'

if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

Download the `FeatureData[Sequence]` from our data which was made in the FirstLook.ipynb:

In [2]:
! wget -nv -O $data_dir/rep-seqs.qza 'https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Sequences_rep_set.qza'

2022-11-09 09:05:42 URL:https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Sequences_rep_set.qza [390624/390624] -> "project_data/rep-seqs.qza" [1]


In [3]:
! qiime metadata tabulate \
    --m-input-file $data_dir/rep-seqs.qza \
    --o-visualization $data_dir/rep-seqs.qzv

Saved Visualization to: project_data/rep-seqs.qzv


In [7]:
Visualization.load(f'{data_dir}/rep-seqs.qzv')

<visualization: Visualization uuid: 4551ef6f-ab23-4594-9933-c842dcc22a62>

Download the `FeatureTable[Frequency]` containing a mapping of the dereplicated sequences to samples from our data which was made in the FirstLook.ipynb:


In [2]:
! wget -nv -O $data_dir/table.qza 'https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Feature_table.qza'

2022-11-03 11:48:13 URL:https://polybox.ethz.ch/index.php/s/MBLSUQXzglnn66u/download?path=%2F&files=Feature_table.qza [504534/504534] -> "project_data/table.qza" [1]


In [5]:
! qiime feature-table summarize \
    --i-table $data_dir/table.qza \
    --o-visualization $data_dir/table.qzv

Saved Visualization to: project_data/table.qzv


In [6]:
Visualization.load(f'{data_dir}/table.qzv')

<visualization: Visualization uuid: 8cbf64f1-2870-457b-94cd-1eb702c56217>

## 2. Taxonomy assignment

To classify the sequences into bacterial species, the assignment can be done with BLAST search of the sequences against a data base of known sequence. The methode used here is a machine learning classifier which is trained on a reference database to recognize the bacterial species in the samples.  


<a id='ref_db'></a>

### 2.1 Reference database construction

The SSU SILVA database (version 138) using the `RESCRIPt` is used in an other notebook Taxonomy_Silva.ipynb. 

In this notebook [a custom database from NCBI sequences using a custom entrez query](https://forum.qiime2.org/t/using-rescript-to-compile-sequence-databases-and-taxonomy-classifiers-from-ncbi-genbank/15947) is build up and cleaned from lower-quality sequences. 

**Citation:** Michael S Robeson II, Devon R O'Rourke, Benjamin D Kaehler, Michal Ziemski, Matthew R Dillon, Jeffrey T Foster, Nicholas A Bokulich. 2021. "RESCRIPt: Reproducible sequence taxonomy reference database management". PLoS Computational Biology 17 (11): e1009581.; [doi: 10.1371/journal.pcbi.1009581](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1009581)

#### 2.1.1 Data download

The sequences and corresponding taxonomies are downloaded with ```RESCRIPt``` for
the NCBI 16S rRNA gene RefSeqs data and save them into respective QIIME 2 artifacts.

The 7-level taxonomy with domain, phylum, class, order, family, genus, species is downloaded to make an accurate prediction of the taxonomy found in the data. 

In [3]:
! qiime rescript get-ncbi-data \
    --p-query '33175[BioProject] OR 33317[BioProject]' \
    --p-n-jobs 3 \
    --o-sequences $data_dir/ncbi-refseqs-unfiltered.qza \
    --o-taxonomy $data_dir/ncbi-refseqs-taxonomy-unfiltered.qza

Saved FeatureData[Sequence] to: project_data/ncbi-refseqs-unfiltered.qza
Saved FeatureData[Taxonomy] to: project_data/ncbi-refseqs-taxonomy-unfiltered.qza


#### 2.1.2 Database curation

To clean up the Database, sequences that contain 5 or more ambigous bases and any homopolymers that are at least 8 bases long are removed from the database. 

In [4]:
! qiime rescript cull-seqs \
    --i-sequences $data_dir/ncbi-refseqs-unfiltered.qza \
    --p-num-degenerates 5 \
    --p-homopolymer-length 8 \
    --p-n-jobs 3 \
    --o-clean-sequences $data_dir/ncbi-refseqs-cleaned.qza

Saved FeatureData[Sequence] to: project_data/ncbi-refseqs-cleaned.qza


Further as some of the sequences of the NCBI 16r are unusually short and might represent truncated 16S rRNA gene sequences. These are filter out to better focus on full-length sequences.
Sequences that are shorter than certain threshold are removed as well from the database. The threshold is specific whether the sequence belong to Archaea or Bacteria.


In [3]:
# NCBI 
! qiime rescript filter-seqs-length-by-taxon \
    --i-sequences $data_dir/ncbi-refseqs-cleaned.qza \
    --i-taxonomy $data_dir/ncbi-refseqs-taxonomy-unfiltered.qza \
    --p-labels Archaea Bacteria \
    --p-min-lens 900 1200 \
    --p-max-lens 1800 2400 \
    --o-filtered-seqs $data_dir/ncbi-refseqs-filt.qza \
    --o-discarded-seqs $data_dir/ncbi-refseqs-tooshort.qza

Saved FeatureData[Sequence] to: project_data/ncbi-refseqs-filt.qza
Saved FeatureData[Sequence] to: project_data/ncbi-refseqs-tooshort.qza


To make sure the taxonomies correspond to the nearly full-length sequences retained. The corresponding taxonomies are filtered out with `filter-taxa`, using the `--m-ids-to-keep-file` parameter (which takes a metadata file, including some artifacts, as input) to only retain features found in our filtered sequences file:

In [6]:
! qiime rescript filter-taxa \
    --i-taxonomy $data_dir/ncbi-refseqs-taxonomy-unfiltered.qza \
    --m-ids-to-keep-file $data_dir/ncbi-refseqs-filt.qza \
    --o-filtered-taxonomy $data_dir/ncbi-refseqs-taxonomy.qza

Saved FeatureData[Taxonomy] to: project_data/ncbi-refseqs-taxonomy.qza


In order to remove identical sequences having the same taxonomies, the database is dereplicated with the uniq mode. In case where identical sequence refere to several taxonmies the recordes are not discarded.

In [8]:
! qiime rescript dereplicate \
    --i-sequences $data_dir/ncbi-refseqs-filt.qza  \
    --i-taxa $data_dir/ncbi-refseqs-taxonomy.qza \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences $data_dir/ncbi-refseqs-derep-uniq.qza \
    --o-dereplicated-taxa $data_dir/ncbi-refseqs-taxonomy-derep-uniq.qza

Saved FeatureData[Sequence] to: project_data/ncbi-refseqs-derep-uniq.qza
Saved FeatureData[Taxonomy] to: project_data/ncbi-refseqs-taxonomy-derep-uniq.qza


#### 2.1.3 PCR-region extraction

The clasiffiers have to be trained on the same type of data as the data that is analyzed in the project in order to get the most accurate classification results. In class the region amplified during NGS library creation out of the full RNA sequence contained in the database is extracted. 


To do this, we will require sequences of both, the forward and reverse, primers used in this experiment - you can look those up in the metadata of this experiment using the [SRA Run Selector](https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=ERP021896&o=acc_s%3Aa). We see the following sequences:

- forward: `GTGCCAGCMGCCGCGGTAA`
- reverse: `GGACTACHVGGGTWTCTAAT`

In [9]:
! qiime feature-classifier extract-reads \
    --i-sequences $data_dir/ncbi-refseqs-derep-uniq.qza \
    --p-f-primer GTGCCAGCMGCCGCGGTAA \
    --p-r-primer GGACTACHVGGGTWTCTAAT \
    --p-n-jobs 3 \
    --p-read-orientation 'forward' \
    --o-reads $data_dir/ncbi-refseqs-515f-806r.qza

Saved FeatureData[Sequence] to: project_data/ncbi-refseqs-515f-806r.qza


The database has to be dereplicated again, because the reads are significantly shorter than at the beginning. The `uniq` mode is choosen again, as some sequences are annotated to different taxonomies.  

In [10]:
! qiime rescript dereplicate \
    --i-sequences $data_dir/ncbi-refseqs-515f-806r.qza \
    --i-taxa $data_dir/ncbi-refseqs-taxonomy-derep-uniq.qza \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences $data_dir/ncbi-refseqs-515f-806r-uniq.qza \
    --o-dereplicated-taxa  $data_dir/ncbi-refseqs-taxonomy-515f-806r-derep-uniq.qza

Saved FeatureData[Sequence] to: project_data/ncbi-refseqs-515f-806r-uniq.qza
Saved FeatureData[Taxonomy] to: project_data/ncbi-refseqs-taxonomy-515f-806r-derep-uniq.qza


<a id='train_classifier'></a>

### 2.2 Training taxonomy classifier

The processed database with the extracted, dereplicated reads is used to train the classifier with the corresponding taxonomies. The Naive Bayes classifier is used as it has shown very good classification results while not being computationally too expensive.

The classifier is trained based on data with known taxonomies as it is in a further step used to predict taxonomy of unknown sequences.

Here training the classifier and evaluate its accuracy is done in one with `evaluate-fit-classifier`:

In [ ]:
#! qiime feature-classifier fit-classifier-naive-bayes \
#   --i-reference-reads $data_dir/ncbi-refseqs-515f-806r-uniq.qza \
#    --i-reference-taxonomy $data_dir/ncbi-refseqs-taxonomy-515f-806r-derep-uniq.qza \
#   --o-classifier $data_dir/ncbi-515f-806r-classifier.qza

In [11]:
! qiime rescript evaluate-fit-classifier \
    --i-sequences $data_dir/ncbi-refseqs-515f-806r-uniq.qza \
    --i-taxonomy $data_dir/ncbi-refseqs-taxonomy-515f-806r-derep-uniq.qza \
    --o-classifier $data_dir/ncbi-refseqs-classifier.qza \
    --o-evaluation $data_dir/ncbi-refseqs-classifier-evaluation.qzv \
    --o-observed-taxonomy $data_dir/ncbi-refseqs-predicted-taxonomy.qza

<a id='tax_assignment'></a>

### 2.3 Taxonomy assignment


After all the preprocessing steps it is time to assign taxonomy labels to the ASVs from the project data. The `classify-sklearn` action from the `feature-classifier` plugin needs two things: 
- the classifier which was trained the previous step
- the sequences to be classified

This step will require the `FeatureData[Sequence]` artifact (containing our ASVs) that were generated beforhand.
To run the following cell at least 10 GB of available RAM is required.

In [11]:
! qiime feature-classifier classify-sklearn \
    --i-classifier $data_dir/ncbi-refseqs-classifier.qza \
    --i-reads $data_dir/rep-seqs.qza \
    --o-classification $data_dir/ncbi-taxonomy.qza

Saved FeatureData[Taxonomy] to: w4_data/taxonomy.qza


A new `FeatureData[Taxonomy]` artifact should be created, containing our taxonomic assignments per feature.

In [12]:
! qiime tools peek $data_dir/ncbi-taxonomy.qza

UUID:        868dff13-8e47-4622-b2ca-531d584726d6
Type:        FeatureData[Taxonomy]
Data format: TSVTaxonomyDirectoryFormat


<a id='tax_visualization'></a>

### 2.4 Taxonomy visualization

The composition of the project samples is in the coming section analyzed. A tabular representation of all the features labeled with their corresponding taxonomy is created:

In [13]:
! qiime metadata tabulate \
    --m-input-file $data_dir/ncbi-taxonomy.qza \
    --o-visualization $data_dir/ncbi-taxonomy.qzv

Saved Visualization to: w4_data/taxonomy.qzv


In the table an ID of every ASV is asigned with its corresponding taxonomic assignment and the prediction confidence.

In [4]:
Visualization.load(f'{data_dir}/ncbi-taxonomy.qzv')

<visualization: Visualization uuid: e3840f9c-beb2-482e-9d6c-41f27717c933>

The taxonomic information per feature can be combined with the information about the samples to get an idea of the taxonomic distribution of species is in the different samples. The data can be visualized in a bar plot. Each bar represents a single sample and is broken down proportionally to counts of every taxon.

In [15]:
! qiime taxa barplot \
    --i-table $data_dir/table.qza \
    --i-taxonomy $data_dir/ncbi-taxonomy.qza \
    --m-metadata-file $data_dir/cleand_sample_meta_data.tsv \
    --o-visualization $data_dir/ncbi-taxa-bar-plots.qzv

Saved Visualization to: w4_data/taxa-bar-plots.qzv


In [5]:
Visualization.load(f'{data_dir}/ncbi-taxa-bar-plots.qzv')

<visualization: Visualization uuid: 2490648e-2a6e-4dd9-aa27-e605badaa500>

Some of the taxonomic assignment are compared to  BLAST for validation. As from the generated ASVs  the BLAST links are equipped there.

In [6]:
Visualization.load(f'{data_dir}/rep-seqs.qzv')

<visualization: Visualization uuid: 2b01d3f6-d1f1-4360-b4d7-5c5bf09e8da4>

Using BLAST as the taxonomic identifier are the same taxonomies observed as with q2-feature-classifier?

Mitochondrial sequences may have to be filtered out of the feature table and sequences. Therefore the `filter-table` and `filter-seqs` action from the `taxa` plugin are used. To exclude features meeting certain criteria we can use the `p-exclude` parameter as follows:

In [18]:
! qiime taxa filter-table \
    --i-table $data_dir/table.qza \
    --i-taxonomy $data_dir/ncbi-taxonomy.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table $data_dir/ncbi-table-filtered.qza

! qiime taxa filter-seqs \
    --i-sequences $data_dir/rep-seqs.qza \
    --i-taxonomy $data_dir/ncbi-taxonomy.qza \
    --p-exclude mitochondria \
    --o-filtered-sequences $data_dir/ncbi-rep-seqs-filtered.qza

Saved FeatureTable[Frequency] to: w4_data/table-filtered.qza
Saved FeatureData[Sequence] to: w4_data/rep-seqs-filtered.qza


The taxa barplot is regenerated using the filtered sequences to compare it to the previous visualization  to check if the distribution on different taxonomic level have changed between samples. 

In [13]:
! qiime feature-classifier classify-sklearn \
    --i-classifier $data_dir/ncbi-refseqs-classifier.qza \
    --i-reads $data_dir/ncbi-rep-seqs-filtered.qza \
    --o-classification $data_dir/ncbi-taxonomy-filtered.qza

Saved FeatureData[Taxonomy] to: w4_data/taxonomy-filtered.qza


In [14]:
! qiime metadata tabulate \
    --m-input-file $data_dir/ncbi-taxonomy-filtered.qza \
    --o-visualization $data_dir/ncbi-taxonomy-filtered.qzv

Saved Visualization to: w4_data/taxonomy-filtered.qzv


In [15]:
Visualization.load(f'{data_dir}/ncbi-taxonomy-filtered.qzv')

<visualization: Visualization uuid: 994ddee0-6cd2-48a9-8890-5aea3e715f01>

In [16]:
! qiime taxa barplot \
    --i-table $data_dir/ncbi-table-filtered.qza \
    --i-taxonomy $data_dir/taxonomy-filtered.qza \
    --m-metadata-file $data_dir/cleand_sample_metad_ata.tsv \
    --o-visualization $data_dir/ncbi-taxa-bar-plots-filtered.qzv

Saved Visualization to: w4_data/taxa-bar-plots-filtered.qzv


In [17]:
Visualization.load(f'{data_dir}/ncbi-taxa-bar-plots-filtered.qzv')

<visualization: Visualization uuid: de2b61a3-9bf3-4738-be8f-e7eb7551f7af>

Maybe not all ASVs are annotated at species level! Some get cut off due to insufficient taxonomic resolution when classifying short sequences (i.e., they have matches to multiple clades).

CHECK FOR THIS!!!